# Task for Today  

***

## Abnormal Heartbeat Classification  

Given *ECG data about patients' hearts*, let's try to detect **abnormal heartbeats** in the data.

We will use a TensorFlow RNN to make our predictions. 

# Getting Started

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
dfs = [pd.read_csv('../input/heartbeat/ptbdb_' + x + '.csv') for x in ['normal', 'abnormal']]

In [ ]:
for df in dfs:
    df.columns = list(range(len(df.columns)))

In [ ]:
data = pd.concat(dfs, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

data = data.rename({187: 'Label'}, axis=1)

In [ ]:
data

# Preprocessing

In [ ]:
y = data['Label'].copy()
X = data.drop('Label', axis=1).copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)

In [ ]:
X_train

In [ ]:
y_train

# Training

In [ ]:
X_train.shape

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

expand = tf.expand_dims(inputs, axis=2)
gru = tf.keras.layers.GRU(256, return_sequences=True)(expand)
flatten = tf.keras.layers.Flatten()(gru)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)


model = tf.keras.Model(inputs=inputs, outputs=outputs)

print(model.summary())

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

# Results

In [ ]:
results = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print("Test Accuracy: {:.2f}%".format(results[1] * 100))
print("     Test AUC: {:.4f}".format(results[2]))

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/MfFe6FbWNSw